In [1]:
import sys
import pandas as pd
import chromadb
from langchain_community.embeddings import XinferenceEmbeddings

sys.path.append("D:\\Users\\pythonWorkSpace\\testPython\\AI-50\\three_stage_project")

from utils.data_load_utils import excel_to_json , str_to_document , get_ids, json_to_str
from langchain_text_splitters import RecursiveCharacterTextSplitter
from chromadb import Documents, EmbeddingFunction, Embeddings

In [2]:
from utils.config_load import load_yaml_file
from langchain_core.documents import Document

In [3]:
# 读取配置
configs = load_yaml_file(file_path="../../config_file/config.yaml")
configs

{'chroma': {'host': 'direct.virtaicloud.com',
  'port': 20994,
  'db_name': 'testDB'},
 'embedding': {'model_uid': 'custom-bge-m3',
  'server_url': 'http://direct.virtaicloud.com:28511'},
 'data_source': {'path': '.././data/test.xlsx'},
 'llm': {'url': 'http://direct.virtaicloud.com:28408/v1',
  'model_name': 'qwen-vl-chat'}}

In [4]:
from model_utils import model_connect_util
url = "http://direct.virtaicloud.com:28408/v1"
url = "http://direct.virtaicloud.com:20326/v1"
url = "http://direct.virtaicloud.com:25933/v1"
model = model_connect_util.get_self_model_connect(base_url=url,api_key="xx",
                                            model_name="qwen-vl-chat",
                                            stream_option=True)
   

d:\TOOL\anaconda3\Lib\site-packages\langchain_core\utils\utils.py:235: UserWarning: WARNING! stream_options is not default parameter.
                stream_options was transferred to model_kwargs.
                Please confirm that stream_options is what you intended.
  warnings.warn(


In [5]:
# chroma_client = chromadb.HttpClient(host="direct.virtaicloud.com", port=20994)

In [6]:
# 连接到Chroma 服务器
chroma_client = chromadb.HttpClient(host=configs["chroma"]["host"], port=configs["chroma"]["port"])

# 创建一个Embedding实例
embed = XinferenceEmbeddings(
    server_url=configs["embedding"]["server_url"], model_uid=configs["embedding"]["model_uid"]
)


In [7]:
# bge服务器总是超时，换小批量转换和插入数据库
class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        embeddings = []
        num =10
        print(f"total:{len(input)}")
        for idx in range(len(input)//num+1):
            if idx % 10 == 0:
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num}")
            try:
                embeddings += embed.embed_documents(input[idx * num: (idx + 1) * num])
            except TimeoutError as  timeout:
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 发生了错误:{timeout}")
                embeds = XinferenceEmbeddings(
                    server_url=configs["embedding"]["server_url"], model_uid=configs["embedding"]["model_uid"]
                )
                embeddings += embeds.embed_documents(input[idx * num: (idx + 1) * num])
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 重试成功")
            except TypeError as ty:
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 发生了错误:{ty}")
                embeds = XinferenceEmbeddings(
                    server_url=configs["embedding"]["server_url"], model_uid=configs["embedding"]["model_uid"]
                )
                embeddings += embeds.embed_documents(input[idx * num: (idx + 1) * num])
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 重试成功")
            except Exception as e:
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 发生了错误:{e}")
                embeds = XinferenceEmbeddings(
                    server_url=configs["embedding"]["server_url"], model_uid=configs["embedding"]["model_uid"]
                )
                embeddings += embeds.embed_documents(input[idx * num: (idx + 1) * num])
                print(f"当前Embedding批次：{idx * num} : {(idx + 1) * num} 重试成功")
        return embeddings

In [8]:
from chromadb import Documents, EmbeddingFunction, Embeddings
# 小批量转换和插入数据库
class MyEmbeddingFunction2(EmbeddingFunction):
    def __init__(self,embed):
        super().__init__()
        self.embed = embed
    def __call__(self, input: Documents) -> Embeddings:
        # embed the documents somehow
        print("------本次开始执行--------")
        embeddings = self.embed.embed_documents(input)
        print("------本次执行成功--------")
        return embeddings

In [9]:
# 获取集合

collection = chroma_client.get_or_create_collection(name="testDB",embedding_function=MyEmbeddingFunction2(embed=embed))

In [10]:
# 读取 Excel 文件
excel_file_path = '../../data/药品说明书数据库_医药数据查询(1).xlsx'
json_output = excel_to_json(excel_file=excel_file_path)

In [ ]:
json_output['Sheet1'][0]

In [28]:
# 转换json数据为Document对象
documents_data = str_to_document(data=json_output['Sheet1'])

In [ ]:
documents_data[0].metadata

In [30]:
data_str = json_to_str(json_output)

In [31]:
# 文本切分
spliter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)


spliter_result = spliter.split_documents(documents=documents_data)

In [ ]:
len(spliter_result)

In [ ]:
spliter_result[1000].page_content

In [18]:
# for idx, chunk in enumerate(spliter_result):
#     print(idx, len(chunk.page_content))
#     print(chunk.page_content)
#     print("-*-" *30 + "\n")

In [34]:
# 生成metadata
metadatas = []

for doc in spliter_result:
    metadatas.append(doc.metadata)

# 生成content
inputs = []
for doc in spliter_result:
    inputs.append(doc.page_content)

# 生成ids
ids = [get_ids() for _ in spliter_result ]

In [ ]:
inputs[221]

In [ ]:
metadatas[221]

### 保存中间切分数据

In [10]:
import numpy as np
import json

file_name = "metadatas"
dict_array_json = [json.dumps(d) for d in metadatas]

metadatas_to_save = np.array(dict_array_json)

np.save(file_name,metadatas_to_save)


file_name = "inputs"
inputs_to_save = np.array(inputs)

np.save(file_name,inputs_to_save)


file_name = "ids"
ids_to_save = np.array(ids)

np.save(file_name,ids_to_save)

### 加载切分的数据，用户断点续传

In [11]:
import numpy as np
import json

new_metadatas = np.load(file="metadatas.npy").tolist()
# 将JSON字符串转换回字典
new_metadatas = [json.loads(j) for j in new_metadatas]

new_inputs = np.load(file="inputs.npy").tolist()


new_ids = np.load(file="ids.npy").tolist()

In [13]:
new_inputs[221]
new_metadatas[221]

{'source': 'https://www.yaozs.com/sms23/', 'title': '氯沙坦钾氢氯噻嗪片'}

In [13]:
# for idx in range(len(inputs)//10+1):
#     my_docs = inputs[idx * 10: (idx + 1) * 10]
#     # embed.embed_documents(inputs[idx * 50: (idx + 1) * 50])
#     print(len(my_docs))

In [ ]:
len(new_inputs)

In [ ]:
len(metadatas)

In [23]:
# 总是超时,切换为小批量插入数据库
# collection.add(documents=inputs,ids=ids,metadatas=metadatas)

## 方式2

In [12]:
now_count = 1000 + 1100 + 3560+300+200+360+200 +200+320+1400+600+495+110+3320+360+200+150+875+60+40+340+720+150+120+300+300+1050+150+650+50+40+90+90+50+300+450+200+3400+900
now_count

In [13]:
#断点续传 1000 + 1100 + 3560 + 300 + 200 +200 +320+1400+600+495+110+3320 + 360+200+150+875+60+40+340+720 +150+120+300+300+1050+150+650+50+90+90
inputs= new_inputs[now_count :50000]
metadatas = new_metadatas[now_count :50000]
ids = new_ids[now_count :50000]

len(inputs) // 50
len(metadatas)// 50
len(ids)

49850

In [14]:
batch_num = 50

for idx in range(len(inputs)//batch_num + 1):
    
    try:
        print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num}")
        current_batch_inputs = inputs[idx * batch_num: (idx + 1) * batch_num]
        current_batch_metadatas = metadatas[idx * batch_num: (idx + 1) * batch_num]
        current_batch_ids = ids[idx * batch_num: (idx + 1) * batch_num]
        collection.add(documents=current_batch_inputs,ids=current_batch_ids,metadatas=current_batch_metadatas)
        print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num}: 插入数据库完成")
    except Exception as  e:
        try:
            print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num} 发生了错误:{e}")
            collection.add(documents=current_batch_inputs,ids=current_batch_ids,metadatas=current_batch_metadatas)
            print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num} 重试成功")
        except Exception as e:
            print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num} 发生了错误:{e}")
            
            # embed = XinferenceEmbeddings(
            #     server_url=configs["embedding"]["server_url"], model_uid=configs["embedding"]["model_uid"]
            # )
            # collection = chroma_client.get_or_create_collection(name="testDB",embedding_function=MyEmbeddingFunction2())
            collection.add(documents=current_batch_inputs,ids=current_batch_ids,metadatas=current_batch_metadatas)
            print(f"当前Embedding批次：{idx * batch_num} : {(idx + 1) * batch_num} 再次重试成功")
    

当前Embedding批次：0 : 50
------本次开始执行--------
当前Embedding批次：0 : 50 发生了错误:HTTPConnectionPool(host='direct.virtaicloud.com', port=28511): Read timed out. (read timeout=None)
------本次开始执行--------
------本次执行成功--------
当前Embedding批次：0 : 50 重试成功
当前Embedding批次：50 : 100
------本次开始执行--------
------本次执行成功--------
当前Embedding批次：50 : 100: 插入数据库完成
当前Embedding批次：100 : 150
------本次开始执行--------


In [ ]:
ids[0]

In [ ]:
# 通过ID获取数据
collection.get(ids='c058f6685b734f4d89ba5e0931950e51')

In [ ]:
query_embed = embed.embed_query("感冒、咳嗽")
results = collection.query(
    query_embeddings=query_embed, # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

In [28]:
def customer_retriever(q:str):
    query_embed = embed.embed_query(q)
    results = collection.query(
        query_embeddings=query_embed, # Chroma will embed this for you
        n_results=1 # how many results to return
    )
    # print(results)
    return results

In [ ]:
customer_retriever("cehsi")